# COMMENTS

In [4]:
# ============================================
# Install & Import Dependencies
# ============================================
# !pip install scipy pandas

import pandas as pd
import numpy as np
from scipy.io import loadmat
from datetime import datetime, timedelta

# ============================================
# Helper Function: MATLAB datenum → datetime
# ============================================
def matlab2datetime(matlab_datenum):
    return datetime.fromordinal(int(matlab_datenum)) \
           + timedelta(days=matlab_datenum % 1) \
           - timedelta(days=366)

# ============================================
# Load .mat Dataset
# ============================================
data = loadmat('/content/NEUSTG_GESLA_19502020.mat')

lat = data['lattg'].flatten()
lon = data['lontg'].flatten()
sea_level = data['sltg']
station_names = [s[0] for s in data['sname'].flatten()]
time = data['t'].flatten()
time_dt = np.array([matlab2datetime(t) for t in time])

# ============================================
# Select Target Stations
# ============================================
SELECTED_STATIONS = [
    'Annapolis', 'Atlantic_City', 'Charleston', 'Washington', 'Wilmington'
]

selected_idx = [station_names.index(st) for st in SELECTED_STATIONS]
selected_names = [station_names[i] for i in selected_idx]
selected_lat = lat[selected_idx]
selected_lon = lon[selected_idx]
selected_sea_level = sea_level[:, selected_idx]  # time × selected_stations

# ============================================
# Build Preview DataFrame
# ============================================
df_preview = pd.DataFrame({
    'time': np.tile(time_dt[:5], len(selected_names)),
    'station_name': np.repeat(selected_names, 5),
    'latitude': np.repeat(selected_lat, 5),
    'longitude': np.repeat(selected_lon, 5),
    'sea_level': selected_sea_level[:5, :].T.flatten()
})

# ============================================
# Print Data Head
# ============================================
print(f"Number of stations: {len(selected_names)}")
print(f"Sea level shape (time x stations): {selected_sea_level.shape}")
df_preview.head()

Number of stations: 5
Sea level shape (time x stations): (622392, 5)


,time,station_name,latitude,longitude,sea_level
0,1950-01-01 00:00:00.000000,Annapolis,38.98328,-76.4816,1.341
1,1950-01-01 00:59:59.999997,Annapolis,38.98328,-76.4816,1.311
2,1950-01-01 02:00:00.000003,Annapolis,38.98328,-76.4816,1.280
3,1950-01-01 03:00:00.000000,Annapolis,38.98328,-76.4816,1.280
4,1950-01-01 03:59:59.999997,Annapolis,38.98328,-76.4816,1.341


In [5]:
# ============================================
# Convert Hourly → Daily per Station
# ============================================
# Convert time to pandas datetime
time_dt = pd.to_datetime(time_dt)

# Build hourly DataFrame for selected stations
df_hourly = pd.DataFrame({
    'time': np.tile(time_dt, len(selected_names)),
    'station_name': np.repeat(selected_names, len(time_dt)),
    'latitude': np.repeat(selected_lat, len(time_dt)),
    'longitude': np.repeat(selected_lon, len(time_dt)),
    'sea_level': selected_sea_level.flatten()
})

# ============================================
# Compute Flood Threshold per Station
# ============================================
threshold_df = df_hourly.groupby('station_name')['sea_level'].agg(['mean','std']).reset_index()
threshold_df['flood_threshold'] = threshold_df['mean'] + 1.5 * threshold_df['std']

df_hourly = df_hourly.merge(threshold_df[['station_name','flood_threshold']], on='station_name', how='left')

# ============================================
# Daily Aggregation + Flood Flag
# ============================================
df_daily = df_hourly.groupby(['station_name', pd.Grouper(key='time', freq='D')]).agg({
    'sea_level': 'mean',
    'latitude': 'first',
    'longitude': 'first',
    'flood_threshold': 'first'
}).reset_index()

# Flood flag: 1 if any hourly value exceeded threshold that day
hourly_max = df_hourly.groupby(['station_name', pd.Grouper(key='time', freq='D')])['sea_level'].max().reset_index()
df_daily = df_daily.merge(hourly_max, on=['station_name','time'], suffixes=('','_max'))
df_daily['flood'] = (df_daily['sea_level_max'] > df_daily['flood_threshold']).astype(int)

# ============================================
# Feature Engineering (3d & 7d means)
# ============================================
df_daily['sea_level_3d_mean'] = df_daily.groupby('station_name')['sea_level'].transform(
    lambda x: x.rolling(3, min_periods=1).mean())
df_daily['sea_level_7d_mean'] = df_daily.groupby('station_name')['sea_level'].transform(
    lambda x: x.rolling(7, min_periods=1).mean())

# Preview
df_daily.head()

,station_name,time,sea_level,latitude,longitude,flood_threshold,sea_level_max,flood,sea_level_3d_mean,sea_level_7d_mean
0,Annapolis,1950-01-01,1.471958,38.98328,-76.4816,2.396988,2.067,0,1.471958,1.471958
1,Annapolis,1950-01-02,1.455417,38.98328,-76.4816,2.396988,2.505,1,1.463687,1.463687
2,Annapolis,1950-01-03,1.841542,38.98328,-76.4816,2.396988,2.536,1,1.589639,1.589639
3,Annapolis,1950-01-04,1.396750,38.98328,-76.4816,2.396988,1.737,0,1.564569,1.541417
4,Annapolis,1950-01-05,1.704333,38.98328,-76.4816,2.396988,2.292,0,1.647542,1.574000


In [6]:
# ============================================
# Build 7-day → 14-day Training Windows
# ============================================
FEATURES = ['sea_level', 'sea_level_3d_mean', 'sea_level_7d_mean']
HIST_DAYS = 7
FUTURE_DAYS = 14

X_train, y_train = [], []

for stn, grp in df_daily.groupby('station_name'):
    grp = grp.sort_values('time').reset_index(drop=True)
    for i in range(len(grp) - HIST_DAYS - FUTURE_DAYS):
        hist = grp.loc[i:i+HIST_DAYS-1, FEATURES].values.flatten()
        future = grp.loc[i+HIST_DAYS:i+HIST_DAYS+FUTURE_DAYS-1, 'flood'].values
        X_train.append(hist)
        y_train.append(future)

X_train = np.array(X_train)
y_train = np.array(y_train)

print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")

X_train shape: (129560, 21)
y_train shape: (129560, 14)


In [7]:
# ============================================
# Select Historical Window (Manual / Random)
# ============================================

# --- Option 1: RANDOM window ---
# np.random.seed(42)
# date_range = pd.date_range(start='1950-01-01', end='2020-12-15')
# hist_start = np.random.choice(date_range)
# hist_end = hist_start + pd.Timedelta(days=6)

# --- Option 2: MANUAL window ---
hist_start = pd.to_datetime('2013-07-21')
hist_end   = pd.to_datetime('2013-07-27')

# Forecast period
test_start = hist_end + pd.Timedelta(days=1)
test_end   = test_start + pd.Timedelta(days=13)

print(f"Historical window: {hist_start.date()} → {hist_end.date()}")
print(f"Forecast window:   {test_start.date()} → {test_end.date()}")

# ============================================
# Build X_test for Selected Window
# ============================================
FEATURES = ['sea_level', 'sea_level_3d_mean', 'sea_level_7d_mean']
X_test = []

for stn, grp in df_daily.groupby('station_name'):
    mask = (grp['time'] >= hist_start) & (grp['time'] <= hist_end)
    hist_block = grp.loc[mask, FEATURES].values.flatten()
    if len(hist_block) == 7 * len(FEATURES):   # ensure full 7-day block
        X_test.append(hist_block)

X_test = np.array(X_test)
print(f"X_test shape: {X_test.shape}  (stations × {7*len(FEATURES)} features)")

Historical window: 2013-07-21 → 2013-07-27
Forecast window:   2013-07-28 → 2013-08-10
X_test shape: (5, 21)  (stations × 21 features)


In [9]:
from xgboost import XGBRegressor
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, matthews_corrcoef
import pickle

# ============================================
# Train 14 XGBoost Models (1 per forecast day)
# ============================================
models = []
for d in range(14):
    model = XGBRegressor(
        n_estimators=100,
        max_depth=5,
        learning_rate=0.1,
        objective='reg:squarederror',
        random_state=42
    )
    model.fit(X_train, y_train[:, d])
    models.append(model)

# Save the model
with open("xgb_models.pkl", "wb") as f:
    pickle.dump(models, f)

# ============================================
# Forecast 14 Days Ahead
# ============================================
y_pred = np.array([m.predict(X_test) for m in models]).T
y_pred_bin = (y_pred > 0.5).astype(int)

# ============================================
# Collect Ground Truth
# ============================================
y_true = []
for stn, grp in df_daily.groupby('station_name'):
    mask = (grp['time'] >= test_start) & (grp['time'] <= test_end)
    vals = grp.loc[mask, 'flood'].values
    if len(vals) == 14:
        y_true.append(vals)
y_true = np.array(y_true)

# ============================================
# Evaluation
# ============================================
y_true_flat = y_true.flatten()
y_pred_flat = y_pred_bin.flatten()

tn, fp, fn, tp = confusion_matrix(y_true_flat, y_pred_flat).ravel()
acc = accuracy_score(y_true_flat, y_pred_flat)
f1 = f1_score(y_true_flat, y_pred_flat)
mcc = matthews_corrcoef(y_true_flat, y_pred_flat)

print("=== Confusion Matrix ===")
print(f"TP: {tp} | FP: {fp} | TN: {tn} | FN: {fn}")
print("\n=== Metrics ===")
print(f"Accuracy: {acc:.3f}")
print(f"F1 Score: {f1:.3f}")
print(f"MCC: {mcc:.3f}")

=== Confusion Matrix ===
TP: 32 | FP: 8 | TN: 24 | FN: 6

=== Metrics ===
Accuracy: 0.800
F1 Score: 0.821
MCC: 0.596
